In [44]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
import datetime
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report
pd.set_option('display.max_columns', None)
import lightgbm as lgb
from sklearn.metrics import log_loss

# svm

In [ ]:
df=pd.read_csv('DATA.csv')
df=df.dropna(axis=0)

In [140]:
y=df['EColi.']
X=df.loc[:, ['up_down', 'curvature', 'inclination', 'tilt_direction', 'altitude', 'disto_river', 'disto_stations', 
             'disto_mainroad', 'disto_syorizyo', 'supply_hours', 'no_water_days', 'total_population ', 'population_served',
             'popu-served', 'number_taps', 'pipelength', 'pipelength_per_pipe', 'served/pipes', '(popu-served)/pipes', 
             'oldest_pipe_age', 'ST', 'RSF', 'FL', 'PF', 'RF', 'ratio']]

In [141]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3,
                                                  random_state=1, stratify=y)
print('Labels counts in y:', np.bincount(y))
print('Lables counts in y_train:', np.bincount(y_train))
print('Lables counts in y_test:', np.bincount(y_test))

Labels counts in y: [ 98 120]
Lables counts in y_train: [68 84]
Lables counts in y_test: [30 36]


In [142]:
X_train_std=(X_train-X_train.mean(axis=0))/X_train.std(axis=0)
X_test_std=(X_test-X_train.mean(axis=0))/X_train.std(axis=0)

def threshold(fare):
    if fare<=0:
        return 0
    else:
        return 1
    
X_train_std['ST']=X_train_std['ST'].apply(threshold)
X_train_std['RSF']=X_train_std['RSF'].apply(threshold)
X_train_std['FL']=X_train_std['FL'].apply(threshold)
X_train_std['PF']=X_train_std['PF'].apply(threshold)
X_train_std['RF']=X_train_std['RF'].apply(threshold)
X_test_std['ST']=X_test_std['ST'].apply(threshold)
X_test_std['RSF']=X_test_std['RSF'].apply(threshold)
X_test_std['FL']=X_test_std['FL'].apply(threshold)
X_test_std['PF']=X_test_std['PF'].apply(threshold)
X_test_std['RF']=X_test_std['RF'].apply(threshold)

In [135]:
from sklearn.svm import SVC
svm=SVC(kernel='linear', C=100, random_state=1, probability=True)

svm.fit(X_train_std, y_train)

SVC(C=100, kernel='linear', probability=True, random_state=1)

In [136]:
svm.score(X_train_std, y_train)

0.7039473684210527

In [137]:
svm.score(X_test_std, y_test)

0.696969696969697

In [115]:
# チューニング
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
pipe_svc=make_pipeline( SVC(random_state=1))

param_range=[0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid=[{'svc__C':param_range, 'svc__kernel':['linear']}]

gs=GridSearchCV(estimator=pipe_svc,
                param_grid=param_grid,
                scoring='accuracy',
                cv=5,
                n_jobs=-1)

In [116]:
gs=gs.fit(X_train_std, y_train)
gs.best_score_

0.5797849462365592

In [117]:
gs.best_params_

{'svc__C': 100.0, 'svc__kernel': 'linear'}

In [118]:
from sklearn.svm import SVC
svm=SVC(kernel='linear', C=100, random_state=1, probability=True)

svm.fit(X_train_std, y_train)

SVC(C=100, kernel='linear', probability=True, random_state=1)

In [119]:
svm.score(X_train_std, y_train)

0.7039473684210527

In [120]:
svm.score(X_test_std, y_test)

0.696969696969697

In [122]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0
SearchMethod = 0
param_range=[0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
                                      
dic={}
for i in range(len(param_range)):
    svm=SVC(kernel='linear', C=param_range[i], random_state=1)
    svm.fit(X_train_std, y_train)
    score1 = svm.score(X_test_std, y_test)
    score2 = svm.score(X_train_std, y_train)
    dic[svm]=score1

    if score1>max_score:
        val=cross_val_score(estimator=svm, X=X_train_std, y=y_train, cv=5)
        val1=np.mean(val)
        print('params: %s, train: %3f, test: %3f, val: %3f' %(svm, score2, score1, val1))

params: SVC(C=0.0001, kernel='linear', random_state=1), train: 0.552632, test: 0.545455, val: 0.552688
params: SVC(C=0.001, kernel='linear', random_state=1), train: 0.552632, test: 0.545455, val: 0.552688
params: SVC(C=0.01, kernel='linear', random_state=1), train: 0.598684, test: 0.560606, val: 0.546237
params: SVC(C=0.1, kernel='linear', random_state=1), train: 0.638158, test: 0.560606, val: 0.578925
params: SVC(kernel='linear', random_state=1), train: 0.677632, test: 0.651515, val: 0.552688
params: SVC(C=10.0, kernel='linear', random_state=1), train: 0.690789, test: 0.696970, val: 0.566452
params: SVC(C=100.0, kernel='linear', random_state=1), train: 0.703947, test: 0.696970, val: 0.579785
params: SVC(C=1000.0, kernel='linear', random_state=1), train: 0.710526, test: 0.696970, val: 0.553333


# random forest

In [123]:
df=pd.read_excel('◎位置情報と対応(水道水) 基準満たしているか否か.xlsx')
df=df.dropna(axis=0)

In [124]:
y=df['EColi.']
X=df.loc[:, ['up_down', 'curvature', 'inclination', 'tilt_direction', 'altitude', 'disto_river', 'disto_stations', 
             'disto_mainroad', 'disto_syorizyo', 'supply_hours', 'no_water_days', 'total_population ', 'population_served',
             'popu-served', 'number_taps', 'pipelength', 'pipelength_per_pipe', 'served/pipes', '(popu-served)/pipes', 
             'oldest_pipe_age', 'ST', 'RSF', 'FL', 'PF', 'RF']]

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3,
                                                  random_state=1, stratify=y)
print('Labels counts in y:', np.bincount(y))
print('Lables counts in y_train:', np.bincount(y_train))
print('Lables counts in y_test:', np.bincount(y_test))

Labels counts in y: [ 98 120]
Lables counts in y_train: [68 84]
Lables counts in y_test: [30 36]


In [24]:
X_train_std=(X_train-X_train.mean(axis=0))/X_train.std(axis=0)
X_test_std=(X_test-X_train.mean(axis=0))/X_train.std(axis=0)

def threshold(fare):
    if fare<=0:
        return 0
    else:
        return 1
    
X_train_std['ST']=X_train_std['ST'].apply(threshold)
X_train_std['RSF']=X_train_std['RSF'].apply(threshold)
X_train_std['FL']=X_train_std['FL'].apply(threshold)
X_train_std['PF']=X_train_std['PF'].apply(threshold)
X_train_std['RF']=X_train_std['RF'].apply(threshold)
X_test_std['ST']=X_test_std['ST'].apply(threshold)
X_test_std['RSF']=X_test_std['RSF'].apply(threshold)
X_test_std['FL']=X_test_std['FL'].apply(threshold)
X_test_std['PF']=X_test_std['PF'].apply(threshold)
X_test_std['RF']=X_test_std['RF'].apply(threshold)

In [25]:
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(max_depth=20, criterion='entropy', n_estimators=40, random_state=1)
forest.fit(X_train_std, y_train)

RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=40,
                       random_state=1)

In [26]:
forest.score(X_train_std, y_train)

1.0

In [27]:
forest.score(X_test_std, y_test)

0.7727272727272727

In [43]:
os.getcwd()

'C:\\Users\\kurokiso'

In [1]:
a=1

In [3]:
type(a)==int

True

In [4]:
a='t'

In [5]:
int(a)

ValueError: invalid literal for int() with base 10: 't'

In [8]:
import numpy as np

In [9]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [11]:
list=['t', 'v']

In [12]:
print(list)

['t', 'v']


In [13]:
print(*list)

t v


In [15]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [21]:
for i in range(10-1):
    print(i)

0
1
2
3
4
5
6
7
8


In [19]:
list

[0, 1, 4, 9, 16]

In [ ]:
for i in np.arange(len(s)-1):
  a=int(s[i])-int(s[i+1])
  if a<0:
    print('douw [%d] %a')
  if a==0:
    print('stay')
  else:
    print('up [%d] %a')